# Drowsiness Detection - Training Data Collection

This notebook will collect training images for drowsiness detection.
- **DROWSY**: Images when I appear drowsy (eyes closed, tired expression)
- **NOT_DROWSY**: Images when I appear alert (eyes open, focused)

The camera will provide a 5-second countdown before capturing images.

In [1]:
import cv2
import os
import time
from pathlib import Path

In [2]:
base_dir = "TRAINING_DATA"
classes = ["DROWSY", "NOT_DROWSY"]

for class_name in classes:
    class_path = os.path.join(base_dir, class_name)
    os.makedirs(class_path, exist_ok=True)
    print(f"Created/Verified directory: {class_path}")

print("\nDirectory structure ready!")

Created/Verified directory: TRAINING_DATA\DROWSY
Created/Verified directory: TRAINING_DATA\NOT_DROWSY

Directory structure ready!


## Find Available Cameras

Depending on the device, we want to know the index of the USB camera to prepare training data and NOT with webcam as the usb camera is being used in inference.

In [3]:
def find_cameras(max_cameras=10):
    """
    Detect all available cameras and display a preview to help identify them.
    """
    available_cameras = []
    
    print("Searching for available cameras...")
    print("="*60)
    
    for i in range(max_cameras):
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)  
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                available_cameras.append(i)
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                print(f"✓ Camera {i}: {width}x{height}")
                
                cv2.putText(frame, f"Camera Index: {i}", (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Resolution: {width}x{height}", (50, 100), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, "Press any key for next camera", (50, 150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                cv2.imshow(f'Camera {i} Preview', frame)
                print(f"  Preview window opened. Check which camera this is.")
                cv2.waitKey(2000)  # Show for 2 seconds
                cv2.destroyAllWindows()
            cap.release()
    
    print("="*60)
    if available_cameras:
        print(f"\nFound {len(available_cameras)} camera(s): {available_cameras}")
    else:
        print("No cameras found!")
    
    return available_cameras

In [4]:
cameras = find_cameras()

Searching for available cameras...
✓ Camera 0: 640x480
  Preview window opened. Check which camera this is.
✓ Camera 0: 640x480
  Preview window opened. Check which camera this is.
✓ Camera 1: 640x480
  Preview window opened. Check which camera this is.
✓ Camera 1: 640x480
  Preview window opened. Check which camera this is.

Found 2 camera(s): [0, 1]

Found 2 camera(s): [0, 1]


<b> Index 1 is for the USB camera

In [7]:
def capture_images(class_name, num_images, camera_index=1, countdown_seconds=5):
    """
    Capture images from USB camera and save them to the specified class folder.
    
    Parameters:
    - class_name: Either "DROWSY" or "NOT_DROWSY"
    - num_images: Number of images to capture
    - camera_index: Camera index (0 for webcam, 1 for USB camera)
    - countdown_seconds: Seconds to wait before starting capture
    """
    
    cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)
    
    if not cap.isOpened():
        print(f"Error: Cannot open camera at index {camera_index}")
        print("Try changing camera_index to 0 if using built-in webcam")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print(f"\n{'='*60}")
    print(f"CAPTURING {num_images} IMAGES FOR CLASS: {class_name}")
    print(f"{'='*60}")
    print(f"\nGet ready! Countdown starting in 2 seconds...")
    time.sleep(2)
    
    for i in range(countdown_seconds, 0, -1):
        ret, frame = cap.read()
        if ret:
            frame_display = frame.copy()
            cv2.putText(frame_display, f"Get Ready: {i}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 3)
            cv2.putText(frame_display, f"Class: {class_name}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Data Collection', frame_display)
            cv2.waitKey(1)
        print(f"Starting in {i}...")
        time.sleep(1)
    
    print("\nStarting capture NOW!")
    
    class_path = os.path.join(base_dir, class_name)
    captured = 0
    
    while captured < num_images:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to capture frame")
            break
        
        timestamp = int(time.time() * 1000) 
        filename = f"{class_name}_{timestamp}_{captured+1}.png"
        filepath = os.path.join(class_path, filename)
        cv2.imwrite(filepath, frame)
        
        frame_display = frame.copy()
        cv2.putText(frame_display, f"Capturing: {captured+1}/{num_images}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 3)
        cv2.putText(frame_display, f"Class: {class_name}", (50, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.imshow('Data Collection', frame_display)
        
        captured += 1
        print(f"Captured {captured}/{num_images}: {filename}")
        
        # Small delay between captures 
        cv2.waitKey(50)  # 50ms delay between captures
    
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"\n✓ Successfully captured {captured} images for {class_name}")
    print(f"  Saved to: {class_path}\n")

# Capture DROWSY Images

In [9]:
num_images = 1000  # Adjust number of images as needed
capture_images("DROWSY", num_images, camera_index=1, countdown_seconds=5)


CAPTURING 1000 IMAGES FOR CLASS: DROWSY

Get ready! Countdown starting in 2 seconds...
Starting in 5...
Starting in 5...
Starting in 4...
Starting in 4...
Starting in 3...
Starting in 3...
Starting in 2...
Starting in 2...
Starting in 1...
Starting in 1...

Starting capture NOW!
Captured 1/1000: DROWSY_1761443225391_1.png
Captured 2/1000: DROWSY_1761443225452_2.png
Captured 3/1000: DROWSY_1761443225516_3.png

Starting capture NOW!
Captured 1/1000: DROWSY_1761443225391_1.png
Captured 2/1000: DROWSY_1761443225452_2.png
Captured 3/1000: DROWSY_1761443225516_3.png
Captured 4/1000: DROWSY_1761443225584_4.png
Captured 5/1000: DROWSY_1761443225647_5.png
Captured 6/1000: DROWSY_1761443225709_6.png
Captured 7/1000: DROWSY_1761443225772_7.png
Captured 4/1000: DROWSY_1761443225584_4.png
Captured 5/1000: DROWSY_1761443225647_5.png
Captured 6/1000: DROWSY_1761443225709_6.png
Captured 7/1000: DROWSY_1761443225772_7.png
Captured 8/1000: DROWSY_1761443225834_8.png
Captured 9/1000: DROWSY_176144322589

# Capture NOT_DROWSY Images

In [10]:
num_images = 1000
capture_images("NOT_DROWSY", num_images, camera_index=1, countdown_seconds=5)


CAPTURING 1000 IMAGES FOR CLASS: NOT_DROWSY

Get ready! Countdown starting in 2 seconds...
Starting in 5...
Starting in 5...
Starting in 4...
Starting in 4...
Starting in 3...
Starting in 3...
Starting in 2...
Starting in 2...
Starting in 1...
Starting in 1...

Starting capture NOW!
Captured 1/1000: NOT_DROWSY_1761443347048_1.png
Captured 2/1000: NOT_DROWSY_1761443347111_2.png
Captured 3/1000: NOT_DROWSY_1761443347174_3.png

Starting capture NOW!
Captured 1/1000: NOT_DROWSY_1761443347048_1.png
Captured 2/1000: NOT_DROWSY_1761443347111_2.png
Captured 3/1000: NOT_DROWSY_1761443347174_3.png
Captured 4/1000: NOT_DROWSY_1761443347252_4.png
Captured 5/1000: NOT_DROWSY_1761443347315_5.png
Captured 6/1000: NOT_DROWSY_1761443347380_6.png
Captured 7/1000: NOT_DROWSY_1761443347444_7.png
Captured 4/1000: NOT_DROWSY_1761443347252_4.png
Captured 5/1000: NOT_DROWSY_1761443347315_5.png
Captured 6/1000: NOT_DROWSY_1761443347380_6.png
Captured 7/1000: NOT_DROWSY_1761443347444_7.png
Captured 8/1000: NOT

## Verify Dataset

Run the cell below to check how many images were collected for each class.

In [11]:
# Verify the collected data
print("="*60)
print("TRAINING DATA SUMMARY")
print("="*60)

for class_name in classes:
    class_path = os.path.join(base_dir, class_name)
    if os.path.exists(class_path):
        num_files = len([f for f in os.listdir(class_path) if f.endswith('.png')])
        print(f"{class_name:15} : {num_files} images")
    else:
        print(f"{class_name:15} : Folder not found")

print("="*60)
print(f"\nDataset location: {os.path.abspath(base_dir)}")

TRAINING DATA SUMMARY
DROWSY          : 1000 images
NOT_DROWSY      : 1000 images

Dataset location: c:\Users\neupa\OneDrive\Desktop\Fall 2025\DATA 690 - JETSON\Code\TRAINING_DATA
